In [1]:
import sqlite3
import pandas as pd

In [2]:
# creates db if it doesn't already exist
connection = sqlite3.connect("../sql/ccg.db")

In [3]:
magicDf = pd.read_csv("../data/dataMagic/completeMagicClean.csv")
magicDf.head()

,name,setCode,setName,language,types,colors,rarity,releaseDate,releaseYear,gameAvailability,avgPrice,currency,providerListing,date,uuid
0,Air Elemental,LEA,Limited Edition Alpha,English,Creature,U,uncommon,1993-08-05,1993,paper,160.26,USD,retail,2025-08-26,27e92f54-0084-57c2-85e5-197e026fab5c
1,Ancestral Recall,LEA,Limited Edition Alpha,English,Instant,U,rare,1993-08-05,1993,paper,20460.00,USD,retail,2025-08-26,1c17ce18-bf3e-558b-9389-632588f93851
2,Animate Artifact,LEA,Limited Edition Alpha,English,Enchantment,U,uncommon,1993-08-05,1993,paper,55.07,USD,retail,2025-08-26,e035e37e-cb8e-5f12-a5db-fe7f927a3457
3,Animate Dead,LEA,Limited Edition Alpha,English,Enchantment,B,uncommon,1993-08-05,1993,paper,528.04,USD,retail,2025-08-26,a5421ca2-32e5-5b31-bc12-979c626fc6eb
4,Animate Wall,LEA,Limited Edition Alpha,English,Enchantment,W,rare,1993-08-05,1993,paper,685.84,USD,retail,2025-08-26,2b304dc1-8d7d-50a7-a310-2d0e5427935f


In [ ]:
cardsDf = magicDf['gameAvailability', 'colors', 'language', 'name', 'rarity', 'setCode', 'uuid']
pricesDf = magicDf['currency', 'date', 'price', 'providerListing', 'uuid']
setDf = magicDf['setName', 'setCode', 'releaseDate', 'releaseYear']

setDF.to_sql('sets', connection, index=False, if_exists='replace')
setDF.to_sql('sets', connection, index=False, if_exists='replace')
setDF.to_sql('sets', connection, index=False, if_exists='replace')

In [ ]:
cardsMagic = pd.read_csv("../data/dataMagic/cardsMagic.csv")
cardsMagic.to_sql("cardsMagic", connection, if_exists="replace", index=False)
pricesMagic = pd.read_csv("../data/dataMagic/pricesMagic.csv")
pricesMagic.to_sql("pricesMagic", connection, if_exists="replace", index=False)
setsMagic = pd.read_csv("../data/dataMagic/setsMagic.csv")
setsMagic.to_sql("setsMagic", connection, if_exists="replace", index=False)

In [5]:
cardsPokemon = pd.read_excel("../data/dataPokemon/cardsPokemon.xlsx")
cardsPokemon.to_sql("cardsPokemon", connection, if_exists="replace", index=False)
groupsPokemon = pd.read_csv("../data/dataPokemon/groupsPokemon.csv")
groupsPokemon.to_sql("groupsPokemon", connection, if_exists="replace", index=False)

210

In [6]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)

,name
0,cardsMagic
1,pricesMagic
2,setsMagic
3,cardsPokemon
4,groupsPokemon
